<CENTER><H1>A hack to get historical tweets</H1></CENTER>


<BR>
    
* This notebook is based on [a previous notebook](https://www.kaggle.com/keitazoumana/get-tweets-about-a-topic-from-twitter-in-5-steps) by @keitazoumana 
* Have a look to this [medium article](https://medium.com/@jcldinco/downloading-historical-tweets-using-tweet-ids-via-snscrape-and-tweepy-5f4ecbf19032) to have an intuition on snscrape

<BR>
    
Be aware that the most of the code will not work here, since it requires you API Tweeter credentials. The pourpose of this notebook is merely didactic.

# Table of Contents

1. [Introduction](#section1)
2. [Loading AP credentials from a json file](#section2)
3. [Perform your query](#section3)

<a id="section1"></a>
# 1. Introduction

The current tweet API (tweepy) doesn't allow queries longer than one week. It is difficult to get tweets with a certain hashtag (a great amount of tweets) with scrapping techniques, because Tweeter removed **/i/search/timeline'** end of the advanced search window. The [snscrape module](https://github.com/JustAnotherArchivist/snscrape) allows one to scrape tweets without the restrictions of Tweepy, but using it programmatically is poorly documented. That's the reason I digged into the code and ofer a solution to include it in your Python code.

In [ ]:
# import necessary modules
import pandas as pd
import numpy as np

In principe, we can get a DataFrame like the [sentiment analysis on Climate change](https://www.kaggle.com/joseguzman/climate-sentiment-in-twitter)

In [ ]:
climate = pd.read_csv('../input/climate-sentiment-in-twitter/Climate_twitter.csv', index_col= 'id')
climate.head()

In [ ]:
pip install tweepy

In [ ]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

<a id="section2"></a>
# 2. Load API credentials from a json file

Use this video to get your API developer. Get your credentials with the [help of this video](https://www.youtube.com/watch?v=PqqXjwoDQiY) and put in a json file like this:

```json
{
    "_comment": "create you account in https://developer.twitter.com/",
    "api_key": "YOUR_API_KEY",
    "api_secret": "YOUR_API_SECRET",
    "access_token": "YOUR_ACCESS_TOKEN,
    "access_secret":"YOUR_ACCESS_SECRET"
}
```


In [ ]:
import tweepy

In [ ]:
def get_credentials(json_file):
    """
    Reads the json file and start the tweeter API.
    Requires a json file with 'api_key', 'api_secret', 
    'access_token' and 'access_secret'
    
    Argument:
    ---------
    json_file (str)
        path to the json file
        
    Returns:
    The tweeter API
    """
    with open(jsonfile) as fp:
        log = json.load(fp)
        auth = tweepy.OAuthHandler(log['api_key'], log['api_secret'])
        auth.set_access_token(log['access_token'], log['access_secret'])
    
    myAPI = tweepy.API(auth, wait_on_rate_limit = True)
    
    return( myAPI)

In [ ]:
import snscrape.modules.twitter as sntwitter

The key now is to use the sntwitter object to obtain tweeter IDs, as follows

```python
mysearch = f'{search_item} -filter:retweets since:{since} until:{until}'
# collect tweet ids!!
myscraper = sntwitter.TwitterSearchScraper(mysearch).get_items()
```

Now, you can create a function that returns a Pandas DataFrame object with the results of your query

<a id="section3"></a>
# 3. Perform your query

In [ ]:

def search(search_item, since = '2020-01-01', until = '2020-12-15', count = 100):
    """
    Searchs the item in tweets and returns a Pandas DataFrame.

    Arguments:
    ----------
    search_item (str)
        
    Example:
    search_term = "#climate+change -filter:retweets" to avoid retweets
        
    """
    mysearch = f'{search_item} -filter:retweets since:{since} until:{until}'
    # collect tweet ids!!
    myscraper = sntwitter.TwitterSearchScraper(mysearch).get_items()
        
    tweets_id = list()
    for i, tweet in enumerate(myscraper):
        if i>count:
            break
        tweets_id.append([tweet.id, tweet.date, tweet.content, tweet.username])

    # Status object from the Tweeter API
    mytweets = list()
    for myid in np.array(tweets_id)[:,0]:
        status = myAPI.get_status(myid, tweet_mode="extended")
        parsed_tweet = dict()
        # date and time of creation
        parsed_tweet['date'] = status.created_at
        # times the tweet is re-tweeted
        parsed_tweet['retweets'] = st1atus.retweet_count
        # which platform used to post
        parsed_tweet['source'] = status.source
        # location where posted
        #parsed_tweet['geo'] = tweet.geo
        # Name of the user posting
        parsed_tweet['author'] = status.user.name
        # number of likes
        parsed_tweet['likes'] = status.favorite_count

        parsed_tweet['id'] = status.user.id
        # tweet posted (without URLs)
        parsed_tweet['text'] = remove_url( status.full_text)
        # Screen name of the USER
        parsed_tweet['twitter_name'] = status.user.screen_name
        # 
        parsed_tweet['location'] = status.user.location
        # if user is verified
        parsed_tweet['verified'] = status.user.verified
        # number of followers
        parsed_tweet['followers'] = status.user.followers_count
        # number of people the user follows
        parsed_tweet['friends'] = status.user.friends_count

        mytweets.append(parsed_tweet)

    df_tweet = pd.DataFrame(mytweets)
    # just in case, remove duplicates
    df_tweet.drop_duplicates('text', keep='first', inplace=True)
    df_tweet.set_index('id', inplace=True)
        
    return(df_tweet)

In [ ]:
search(search_item = "#climate+change -filter:retweets', since = '2020-01-01', until = '2020-12-15', count = 1000):